In [1]:
import pandas as pd
import glob
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import glob
import docx

def preprocess_file (path): 
    files = glob.glob("C:\\Users\\ncy_k\\Desktop\\EMBRACE Redacted Transcripts\\Compiled Data\\" + path)

    para_list = []
    tmp_list = []
    include = False

    for file in files:
        paras = docx.Document(file).paragraphs
        include = False
        if paras != None:
            for para in paras:
                para = para.text
                colon_index = para.find(":")
                speaker = para[0:colon_index]
                if colon_index != -1 and (speaker == "LD" or speaker == "LD2" or speaker == "LD1"):
                    continue
                para = para[colon_index+1:].strip().lower()
                for word in para:
                    if word.isdigit():
                        para = para.replace(word,"")
                if para.replace(":","").isdigit():
                    continue
                if len(para)>0:
                    para_list.append(para)
    return para_list

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import numpy as np
import tqdm

import gensim
#measure topic coherence and opacity. Plot topic coherence measures with number of topics until it reaches max/plateau
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ncy_k\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ncy_k\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load("Google-News-Vectors")

In [5]:
#tokenize and remove stop words by sentence
def tokenize_remove_stopwords (para_list):
    stopwords_list = ["ok", 'u', "1", "one", "yea", "2", "u", "oh", "also", "yeah", "00", "okay", "", "hm", "haha", "5l", "mm", 
                    "k", "b", "le", "u", "lo", "p", "unintelligible"]
    #tokenize and remove stop words by sentence
    all_sw = stopwords.words('English')
    all_sw.extend(stopwords_list)
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    wordnet_lemmatizer = WordNetLemmatizer()

    tokenized_and_cleaned_doc = []
    print (f"Length: {len(para_list)}")
    for i in para_list:
        tokens = tokenizer.tokenize(i)
        tokens_without_sw = []
        tokens_without_sw = [wordnet_lemmatizer.lemmatize(word.strip()) for word in tokens if len(word) > 3 and not (word.strip() in all_sw)]
        if len(tokens_without_sw) > 0: 
            tokenized_and_cleaned_doc.append(tokens_without_sw)

    print(f"{len(tokenized_and_cleaned_doc)} documents generated.")
    return tokenized_and_cleaned_doc

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')

filepaths = ["Compiled Data Challenges.docx", "Compiled Data Challenges - Volunteer.docx", "Compiled Data Challenges - SAC Staff.docx"]
filenames = ["Challenges-Full", "Challenges-Volunteers", "Challenges-SAC Staff"]
index = 0

for path in filepaths:
    para_list = preprocess_file(path)
    tokenized_and_cleaned_doc = tokenize_remove_stopwords(para_list)
    file = open("LFDLA-" + filenames[index] +".txt", 'a', encoding="utf-8")
    for token in tokenized_and_cleaned_doc: 
        if len(token) > 0 and not (token is None):
            file.write(" ".join(token) + "\n")
    file.close()
    index+=1


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ncy_k\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ncy_k\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Length: 516
403 documents generated.
Length: 271
185 documents generated.
Length: 89
79 documents generated.
